## 类Beam(object):
### 变量
- size: = beamSize
- done: 默认false 用处目前不明 ？？？？
- tt: = torch.cuda if cuda else torch
- scores: beam中各个翻译的分数 初始化为self.tt.FloatTensor(size).zero_()
- allScores: 列表类 用处目前不明 ？？？？
- prevKs: 列表类 记录各个时间点的prevK。prevK指明了当前单词是来自哪个beam
- nextYs: 各个时间节点的输出 初始化为[self.tt.LongTensor(size).fill_(onmt.Constants.PAD)]，nextYs[0][0] = onmt.Constants.BOS。初始只有一项
- attn: 列表类，记录各个时间节点上的attention

### 方法
##### __init__(self, size, cuda = False):
- size是beamSize
- 主要是对变量进行初始化

##### getCurrentState(self):
- 获得当前时间节点的输出
- out size: beam_size
- return self.nextYs[-1]

##### getCurrentOrigin(self):
- 获得当前时间节点的backpointer
- out size: beam_size
- return self.prevKs[-1]

##### advance(self, wordLK, attnOut):
- 两个参数:
    - wordLk: (beamSize, tgt_dict.size)
    - attn: (beamSize, seq_len)
- 返回：True if beam search is complete
- 主要实现，把generator得到的分数和历史分数进行整合，选择合适的单词，作为各个beam的当前的候选单词。并记录起来。
- 后面还是看代码把：

In [ ]:
def advance(self, wordLk, attnOut):
    # 这里就先当numWords时tgt_dict.size了
    numWords = wordLk.size(1)
    
    # prevKs不知道是什么鬼？？？
    if len(self.prevKs) > 0:
        # score是[beam]的，保存着上一轮每个beam的得分。加上历史得分，从而得到目前为止总的分数
        beamLk = wordLk + self.scores.unsqueeze(1).expand_as(wordLk)
    else:
        beamLk = wordLk[0]
    
    # flat后进行排序是为了从整体中得到最优的点。假如我们从各个beam的候选单词中各自选出一个对应最优分数的单词，那么也就是说，一旦确定了，
    # 这个选择是不能改变的。后来者都会用到这个选择。就像贪婪算法，这并不能得到全局最优。flat后的方法，虽然也不能得到全局最优，
    # 但至少，它多了(beamSize-1)*numWords个选择,
    # 举个例子，字母表是a, b, c。 beamSize是2.首轮a, b分数最高分别是3，1。第二轮得到的a, b, c的得分分别为1, 2, 1.
    # 那么扩展相加后的得分是:[[4, 5, 4], [2, 3, 2]]。
    # 如果按照每个beam选一个那么这里会选的是1b,2b。即得到的句子为ab(4), bb(3)
    # 如果按照flat后进行选择，则这里选的会是1a,1b。即得到的句子为aa(4), ab(5)
    # 可以明显看到第二种方法的结果更优。这所以说这里不是全局最优，如上面可以看到，第二轮的选择上，已经不再注意首轮选c的情况了。
    flatBeamLk = beamLk.view(-1)
    bestScores, bestScoresId = flatBeamLk.topk(self.size, 0, True, True)
    self.allScores.appen(self.scores)
    self.scores = bestScores
    
    # bestScoresId is flattended beam X word array, so calculate which
    # word and beam each score came from 
    # 正如相面说的topk是从flat后的数据中选择的，这里我们想具体的知道，这个被选中的家伙是从哪个beam中被选出来的。从而可以把他添加到对应
    # 的历史数据中。
    # 下面这个语句，可以看出这个单词是来自哪个beam
    prevK = bestScoresId /numWords
    self.prevKs.append(prevK)
    self.nextYs.append(bestScoresId - prevK * numWords)
    # 记录attn
    self.attn.append(attnOut.index_select(0, prevK))
    
    # End condition is when top-of-beam is EOS
    if self.nextYs[-1][0] == onmt.Constants.EOS:
        self.done = True
        self.allScores.append(self.scores)
    
    return self.done

##### sortBest(self):
- return torch.sort(self.scores, 0, True)

##### getBest(self):
- get the score of the best in the beam

##### getHyp(self, k):
- 获得beamSize个预测的分数最高的句子
- prevK指向了当前单词使用的历史对象，而他又可以从上一个事件点的nextY中直接提取出来，所以从后往前依次提取出来就是了。

In [ ]:
def getBest(self):
    hyp, attn = [], []
    for j in range(len(self.prevKs) - 1, -1, -1):
        hyp.append(self.nextYs[j + 1][k])
        attn.append(self.attn[j][k])
        k = self.prevKs[j][k]
    
    return hyp[::-1], torch.stack(attn[::-1])